In [1]:
import sys
sys.path.append("./../")
from evaluation.gsm8k import judge
import json
from thirdparty.openai.grade_school_math.dataset import read_jsonl
import numpy as np
from scipy.stats import bootstrap
import os

In [2]:
reference = read_jsonl("./../data/gsm8k/test.json")

In [3]:
def clean(text):
    texts = text.split("####")
    if len(texts) == 1:
        return text
    last_txt = "#### " + "".join(texts[-1].split(" "))
    return "####".join(texts[:-1]) + last_txt


names = os.listdir("./../result")
name2data = {}
for name in names:
    with open(f"./../result/{name}", "r") as f:
        data = json.load(f)
        
    for entry in data:
        if "clean_output" in entry:
            entry["answer"] = clean(entry["clean_output"])
        else:
            entry["answer"] = entry["output"]
    name2data[name] = data

In [4]:
reses = []
for name, data in name2data.items():
    
    if len(data) != len(reference) or name == "mistral_v1_outputs_train.json":
        continue
    results = judge(data, reference)
    
    results = np.array(results).reshape((1,-1))
    
    ci = bootstrap(results, np.mean).confidence_interval
        
    reses.append((f"{name}: {np.mean(results)}, {ci}, {len(data)} samples", results))

reses.sort(key= lambda x: x[1].mean(),reverse = True)
for res,val in reses:
    print(res)

mistral_v1_outputs_test.json: 0.3813495072024261, ConfidenceInterval(low=0.3555724033358605, high=0.40788476118271416), 1319 samples
mistral_v1_outputs_test_pause_token.json: 0.3639120545868082, ConfidenceInterval(low=0.33813495072024263, high=0.3912054586808188), 1319 samples
ll_reward_3_samps_rollout_normal_sft_friday_outer_loop_5.json: 0.35178165276724793, ConfidenceInterval(low=0.3260045489006823, high=0.3775587566338135), 1319 samples
ll_reward_3_samps_rollout_normal_sft_friday_outer_loop_3.json: 0.3502653525398029, ConfidenceInterval(low=0.3244882486732373, high=0.37604245640636846), 1319 samples
ll_reward_3_samps_rollout_normal_sft_friday_outer_loop_2.json: 0.34950720242608035, ConfidenceInterval(low=0.3237300985595148, high=0.3752843062926459), 1319 samples
ll_reward_3_samps_rollout_normal_sft_friday_outer_loop_7.json: 0.34268385140257773, ConfidenceInterval(low=0.31842304776345715, high=0.3694504998952325), 1319 samples
ll_reward_3_samps_rollout_normal_sft_friday_outer_loop_4.

In [23]:
name = "ll_reward_3_samps_rollout_normal_sft_include_gt_beta10_outer_loop_4.json"
name2 = "wsft_pause_constrained_ll_reward_2_samps_rollout_0.json"
index = 80
data = name2data[name]
sample = name2data[name][index]
sample2 = name2data[name2][index]
ref_sample = reference[index]
print("#"*100)
print("question:")
print(ref_sample["question"])
print("#"*100)
print("Ground Truth")
print(ref_sample["answer"])
print("#"*100)
print("Prediction round 1")
print(sample["output"])
print("#"*100)
print("Clean round 1")
print(sample["clean_output"])
print("#"*100)
print(judge(data, reference)[index])
# print("Prediction round 0")
# print(sample2["output"])

####################################################################################################
question:
Two girls each got 1/6 of the 24 liters of water. Then a boy got 6 liters of water. How many liters of water were left?
####################################################################################################
Ground Truth
Each of the girls got 24 x 1/6 = <<24*1/6=4>>4 liters of water.
So the two girls got a total of 4 x 2 = <<4*2=8>>8 liters.
Thus, a total of 8 + 6 = <<8+6=14>>14 liters of water were gotten by the two girls and the boy.
Therefore, 24 - 14 = <<24-14=10>>10 liters of water were left.
#### 10
####################################################################################################
Prediction round 1
<|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|pause|><|

In [108]:
# ll_name2data = list(
#     filter(
#         lambda x: "ll_reward_3_samps" in x,
#         name2data
#     )
# )
# ll_name2data = {k: name2data[k] for k in ll_name2data}
ll_name2data = name2data
name1 = "ll_reward_3_samps_rollout_normal_sft_friday_outer_loop_5.json"
name2 = "mistral_v1_outputs_test_pause_token.json"
judge1 = judge(ll_name2data[name1], reference)
judge2 = judge(ll_name2data[name2], reference)
j1_c = 0
j2_c = 0
for index,(j1,j2) in enumerate(zip(judge1,judge2)):
    if j1 and not j2:
        j1_c +=1
    if j2 and not j1:
        j2_c += 1
    # if j1 and not j2:
    #     print("#"*100)
    #     print(name1, "correct")
    #     print("-"*100)
    #     print("-"*100)
    #     print(ll_name2data[name1][index]["output"])
    #     print("-"*100)
    #     print("+"*100)
    #     print(name2, "inccorrect")
    #     print("-"*100)
    #     print("-"*100)
    #     print(ll_name2data[name2][index]["output"])
    #     print("-"*100)
    #     print("#"*100)
print(name1, "samples exculisively correct", j1_c)
print(name2, "samples exculisively correct", j2_c)

ll_reward_3_samps_rollout_normal_sft_friday_outer_loop_5.json samples exculisively correct 177
mistral_v1_outputs_test_pause_token.json samples exculisively correct 193


In [42]:
def sf(text):
    texts = text.split("####")
    if len(texts) == 1:
        return text
    last_txt = "#### " + "".join(texts[-1].split(" "))
    return "####".join(texts[:-1]) + last_txt
for dp in data:
    dp["answer"] = sf(dp["answer"])
    

In [47]:
data[index]["answer"]

' He produces 252 eggs x 7 days = <<252*7 =176 4 >>1764 eggs\nThat means he sells 1764 eggs / 12 = << 1764/12=147>>147 dozen\nHe then makes 14 7 dozen x $2/dozen = $<<1 47*2=294 >> 294 in the week\n#### 294'